In [15]:
# Chatbot Telegram MVP V3: Auto-Idioma, Conciso, Voz e Lembretes

In [21]:
!pip install -q python-telegram-bot --upgrade google-generativeai pyttsx3 gTTS SpeechRecognition pydub > /dev/null
!apt-get update > /dev/null
!apt-get install -y ffmpeg espeak > /dev/null
print("Dependências instaladas.")

import os
import asyncio
import re
import random
import datetime
import pytz
from google.colab import userdata
import google.generativeai as genai
from telegram import Update, ForceReply
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
from telegram.constants import ParseMode
import speech_recognition as sr
from gtts import gTTS
from pydub import AudioSegment

API_KEY_CONFIGURED = False
BOT_TOKEN_CONFIGURED = False

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    API_KEY_CONFIGURED = True
    print("INFO: Chave API Gemini OK.")
except Exception as e:
    print(f"ERRO: Chave API Gemini - {e}")

try:
    TELEGRAM_BOT_TOKEN = userdata.get('TELEGRAM_BOT_TOKEN')
    if not TELEGRAM_BOT_TOKEN: raise ValueError("Token do Telegram vazio.")
    BOT_TOKEN_CONFIGURED = True
    print("INFO: Token Telegram OK.")
except Exception as e:
    print(f"ERRO: Token Telegram - {e}")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Dependências instaladas.
INFO: Chave API Gemini OK.
INFO: Token Telegram OK.


In [20]:
chat_histories = {} # {chat_id: [historico]}
stt_recognizer = sr.Recognizer()

async def get_gemini_response_v3(chat_id: int, user_input_text: str, detected_input_lang: str):
    if not API_KEY_CONFIGURED: return "AI Key missing.", None, "en"
    if not user_input_text: return "Input was empty.", None, detected_input_lang

    history = chat_histories.get(chat_id, [])
    response_lang_code_for_tts = detected_input_lang
    prompt_parts, tts_text_override = [], None

    if detected_input_lang == "pt":
        response_lang_code_for_tts = "pt"
        prompt_parts.append(f"""Você é 'English Buddy PT', tutor de inglês AI CONCISO no Telegram. Usuário (PT): "{user_input_text}"
Responda em PORTUGUÊS, CURTO e SIMPLES:
1. Pergunta sobre tradução para inglês: dê a tradução direta e UM exemplo CURTO em inglês com tradução. EVITE gramática longa.
2. Dúvida geral: responda CURTO e útil.
3. Comentário: reconheça BREVEMENTE, se apropriado, sugira algo similar em inglês.
Exemplo para "Como se diz 'estou com sono' em inglês?":
"Em inglês, 'estou com sono' é 'I'm sleepy'. Ex: 'I'm sleepy, I think I'll go to bed.' (Estou com sono, acho que vou para a cama.)"
Sua resposta (CURTA, SIMPLES, PT):""")
    else: # 'en'
        response_lang_code_for_tts = "en"
        prompt_parts.append(f"""You are 'English Buddy', a CONCISE AI English tutor on Telegram. User (EN): "{user_input_text}"
Respond in ENGLISH, SHORT and SIMPLE:
1. Analyze for CRITICAL errors.
2. If CRITICAL error: provide corrected version and a VERY BRIEF tip (1-2 short sentences). Format: User: "..." / Corrected: "..." / Tip: "..."
3. If understandable: respond NATURALLY and BRIEFLY. Avoid minor corrections.
4. Meaning request: SHORT definition, ONE example.
Ex for "I has a cat.":
"User: "I has a cat."
Corrected: "I *have* a cat."
Tip: Nice! For 'I', we use 'have'."
Your response (SHORT, SIMPLE, EN):""")

    try:
        model = genai.GenerativeModel(model_name='gemini-2.0-flash')
        current_turn_history = list(history)
        current_turn_history.append({'role': 'user', 'parts': [user_input_text]})

        MAX_HISTORY_TURNS = 5
        if len(current_turn_history) > MAX_HISTORY_TURNS * 2:
            current_turn_history = current_turn_history[-(MAX_HISTORY_TURNS * 2):]


        final_payload_for_gemini = [{'role': 'user', 'parts': prompt_parts}]



        gen_response = await asyncio.to_thread(
            model.generate_content, final_payload_for_gemini,
            generation_config=genai.types.GenerationConfig(max_output_tokens=150, temperature=0.65)
        )
        response_text_for_user = gen_response.parts[0].text.strip() if gen_response.parts else ""
        if not response_text_for_user: return "AI response was empty.", None, detected_input_lang

        if response_lang_code_for_tts == "en" and "Corrected:" in response_text_for_user:
            match = re.search(r"Corrected:\s*\"(.*?)\"", response_text_for_user, re.DOTALL | re.IGNORECASE)
            if match: tts_text_override = match.group(1).replace("*", "")
        elif response_lang_code_for_tts == "pt":
             match_en_in_pt = re.search(r"Em inglês, [^']*?\s*é\s*'([^']*)'", response_text_for_user, re.IGNORECASE)
             if match_en_in_pt:
                 tts_text_override = match_en_in_pt.group(1)
                 response_lang_code_for_tts = "en"

        history.append({'role': 'user', 'parts': [user_input_text]})
        history.append({'role': 'model', 'parts': [response_text_for_user]})
        if len(history) > MAX_HISTORY_TURNS * 2: history = history[-(MAX_HISTORY_TURNS * 2):]
        chat_histories[chat_id] = history
        return response_text_for_user, tts_text_override, response_lang_code_for_tts
    except Exception as e:
        print(f"Gemini Error (ChatID {chat_id}): {e}")
        return "AI technical hiccup. Try again shortly.", None, detected_input_lang

async def text_to_speech_v3(text: str, chat_id: int, lang_code_and_tld: str = 'en-US', filename_prefix: str = "tts_audio_v3"):
    try:
        lang_short = lang_code_and_tld.split('-')[0]
        tld_map = {'en-US': 'com', 'en-GB': 'co.uk', 'pt-BR': 'com.br'}
        tld = tld_map.get(lang_code_and_tld, 'com' if lang_short == 'en' else 'com.br')
        fname = f"{filename_prefix}_{chat_id}_{lang_short}_{tld.replace('.','')}_{int(time.time()*1000)}.mp3"
        gtts_obj = gTTS(text=text, lang=lang_short, tld=tld, slow=False)
        await asyncio.to_thread(gtts_obj.save, fname)
        return fname
    except Exception as e:
        print(f"gTTS Error (ChatID {chat_id}, lang_tld {lang_code_and_tld}): {e}")
        return None

async def speech_to_text_v3(voice_ogg_path: str, chat_id: int):
    base, _ = os.path.splitext(voice_ogg_path)
    temp_wav_path = f"{base}_{chat_id}_temp.wav"
    text_en, text_pt = "", ""
    try:
        audio = await asyncio.to_thread(AudioSegment.from_ogg, voice_ogg_path)
        await asyncio.to_thread(audio.export, temp_wav_path, format="wav")
        with sr.AudioFile(temp_wav_path) as source: audio_data = stt_recognizer.record(source)
        try: text_en = await asyncio.to_thread(stt_recognizer.recognize_google, audio_data, language="en-US")
        except: pass
        try: text_pt = await asyncio.to_thread(stt_recognizer.recognize_google, audio_data, language="pt-BR")
        except: pass

        len_en, len_pt = len(text_en.split()), len(text_pt.split())
        if len_en > 0 and len_pt == 0: return text_en, "en"
        if len_pt > 0 and len_en == 0: return text_pt, "pt"
        if len_en > 0 and len_pt > 0: return (text_pt, "pt") if len_pt > len_en + 1 else (text_en, "en")
        return "", None
    except Exception as e:
        print(f"STT Error (ChatID {chat_id}, File {voice_ogg_path}): {e}")
        return "STT processing error.", None
    finally:
        if os.path.exists(temp_wav_path): os.remove(temp_wav_path)

In [19]:
import os
import asyncio
import re
import random
import datetime
import pytz
from google.colab import userdata
import google.generativeai as genai
from telegram import Update, ForceReply
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
from telegram.constants import ParseMode
import speech_recognition as sr
from gtts import gTTS
from pydub import AudioSegment


API_KEY_CONFIGURED = False
try:

    if 'GOOGLE_API_KEY' in os.environ:
        genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
        API_KEY_CONFIGURED = True
    else:
        GOOGLE_API_KEY_LOCAL = userdata.get('GOOGLE_API_KEY')
        genai.configure(api_key=GOOGLE_API_KEY_LOCAL)
        os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY_LOCAL
        API_KEY_CONFIGURED = True
        print("INFO (Célula 3): Chave API Gemini OK.")
except Exception as e:
    print(f"ERRO (Célula 3): Chave API Gemini - {e}")


chat_histories = {}
stt_recognizer = sr.Recognizer()

async def get_gemini_response_v4(chat_id: int, user_input_text: str, detected_input_lang: str):
    """
    Gera resposta do Gemini, adaptando-se ao idioma, focando em concisão E ENGAJAMENTO.
    Retorna (str: texto_da_resposta_para_usuario, str: texto_tts_se_diferente, str: codigo_idioma_resposta_tts)
    """
    if not API_KEY_CONFIGURED: return "My AI brain is not connected (API Key missing).", None, "en"
    if not user_input_text: return "Input was empty.", None, detected_input_lang

    history = chat_histories.get(chat_id, [])
    response_lang_code_for_tts = detected_input_lang
    prompt_parts, tts_text_override = [], None


    engagement_instruction_pt = "\n\nApós sua resposta principal, FAÇA UMA PERGUNTA aberta e relevante para continuar a conversa em português ou convide o usuário a tentar algo novo."
    engagement_instruction_en = "\n\nAfter your main response, ASK an open-ended and relevant question to continue the conversation in English or invite the user to try something new."

    if detected_input_lang == "pt":
        response_lang_code_for_tts = "pt"
        prompt_parts.append(f"""Você é 'English Buddy PT', tutor de inglês AI CONCISO e ENGAJADOR no Telegram. Usuário (PT): "{user_input_text}"
Responda em PORTUGUÊS, CURTO e SIMPLES:
1. Pergunta sobre tradução para inglês: dê a tradução direta e UM exemplo CURTO em inglês com tradução. EVITE gramática longa.
2. Dúvida geral: responda CURTO e útil.
3. Comentário: reconheça BREVEMENTE e, se apropriado, sugira algo similar em inglês.
{engagement_instruction_pt}

Exemplo para "Como se diz 'estou com sono' em inglês?":
"Em inglês, 'estou com sono' é 'I'm sleepy'. Por exemplo: 'I'm sleepy, I think I'll go to bed.' (Estou com sono, acho que vou para a cama.) Que outras frases você gostaria de saber?"

Sua resposta (CURTA, SIMPLES, ENGAJADORA, PT):""")
    else: # 'en'
        response_lang_code_for_tts = "en"
        prompt_parts.append(f"""You are 'English Buddy', a CONCISE and ENGAGING AI English tutor on Telegram. User (EN): "{user_input_text}"
Respond in ENGLISH, SHORT and SIMPLE:
1. Analyze for CRITICAL errors.
2. If CRITICAL error: provide corrected version and a VERY BRIEF tip (1-2 short sentences). Format: User: "..." / Corrected: "..." / Tip: "..."
3. If understandable: respond NATURALLY and BRIEFLY.
4. Meaning request: SHORT definition, ONE example.
{engagement_instruction_en}

Example for "I has a cat.":
"User: "I has a cat."
Corrected: "I *have* a cat."
Tip: Nice! For 'I', we use 'have'. What other sentences are you practicing?"

Your response (SHORT, SIMPLE, ENGAGING, EN):""")

    try:
        model = genai.GenerativeModel(model_name='gemini-2.0-flash')


        payload_for_gemini = list(history)
        payload_for_gemini.append({'role': 'user', 'parts': prompt_parts})

        MAX_HISTORY_TURNS = 5
        if len(payload_for_gemini) > (MAX_HISTORY_TURNS * 2) + 1:

            num_to_remove = len(payload_for_gemini) - ((MAX_HISTORY_TURNS * 2) + 1)
            payload_for_gemini = payload_for_gemini[num_to_remove:]

        print(f"DEBUG Gemini V4 Req: ChatID {chat_id}, InLang {detected_input_lang}, RespLang {response_lang_code_for_tts}, PayloadLen {len(payload_for_gemini)}")

        gen_response = await asyncio.to_thread(
            model.generate_content, payload_for_gemini,
            generation_config=genai.types.GenerationConfig(max_output_tokens=180, temperature=0.7)
        )
        response_text_for_user = gen_response.parts[0].text.strip() if gen_response.parts else ""
        if not response_text_for_user: return "AI response was empty.", None, detected_input_lang


        if response_lang_code_for_tts == "en" and "Corrected:" in response_text_for_user:
            match = re.search(r"Corrected:\s*\"(.*?)\"", response_text_for_user, re.DOTALL | re.IGNORECASE)
            if match: tts_text_override = match.group(1).replace("*", "")
        elif response_lang_code_for_tts == "pt":
             match_en_in_pt = re.search(r"Em inglês, [^']*?\s*é\s*'([^']*)'", response_text_for_user, re.IGNORECASE)
             if match_en_in_pt:
                 tts_text_override = match_en_in_pt.group(1)
                 response_lang_code_for_tts = "en"

        history.append({'role': 'user', 'parts': [user_input_text]})
        history.append({'role': 'model', 'parts': [response_text_for_user]})
        if len(history) > MAX_HISTORY_TURNS * 2: history = history[-(MAX_HISTORY_TURNS * 2):]
        chat_histories[chat_id] = history

        return response_text_for_user, tts_text_override, response_lang_code_for_tts
    except Exception as e:
        print(f"Gemini Error V4 (ChatID {chat_id}): {e}")
        return "AI technical hiccup. Please try again shortly.", None, detected_input_lang


async def text_to_speech_v3(text: str, chat_id: int, lang_code_and_tld: str = 'en-US', filename_prefix: str = "tts_audio_v3"):
    try:
        lang_short = lang_code_and_tld.split('-')[0]
        tld_map = {'en-US': 'com', 'en-GB': 'co.uk', 'pt-BR': 'com.br'}
        tld = tld_map.get(lang_code_and_tld, 'com' if lang_short == 'en' else 'com.br')
        fname = f"{filename_prefix}_{chat_id}_{lang_short}_{tld.replace('.','')}_{int(time.time()*1000)}.mp3"
        gtts_obj = gTTS(text=text, lang=lang_short, tld=tld, slow=False)
        await asyncio.to_thread(gtts_obj.save, fname)
        return fname
    except Exception as e:
        print(f"gTTS Error V3 (ChatID {chat_id}, lang_tld {lang_code_and_tld}): {e}")
        return None

async def speech_to_text_v3(voice_ogg_path: str, chat_id: int):
    base, _ = os.path.splitext(voice_ogg_path)
    temp_wav_path = f"{base}_{chat_id}_temp.wav"
    text_en, text_pt = "", ""
    try:
        audio = await asyncio.to_thread(AudioSegment.from_ogg, voice_ogg_path)
        await asyncio.to_thread(audio.export, temp_wav_path, format="wav")
        with sr.AudioFile(temp_wav_path) as source: audio_data = stt_recognizer.record(source)
        try: text_en = await asyncio.to_thread(stt_recognizer.recognize_google, audio_data, language="en-US")
        except: pass
        try: text_pt = await asyncio.to_thread(stt_recognizer.recognize_google, audio_data, language="pt-BR")
        except: pass

        len_en, len_pt = len(text_en.split()), len(text_pt.split())
        if len_en > 0 and len_pt == 0: return text_en, "en"
        if len_pt > 0 and len_en == 0: return text_pt, "pt"
        if len_en > 0 and len_pt > 0: return (text_pt, "pt") if len_pt > len_en + 1 else (text_en, "en")
        return "", None
    except Exception as e:
        print(f"STT Error V3 (ChatID {chat_id}, File {voice_ogg_path}): {e}")
        return "STT processing error.", None
    finally:
        if os.path.exists(temp_wav_path): os.remove(temp_wav_path)

In [22]:


schedulable_chat_ids_v4 = set()
reminder_messages_audio_en_v4 = [
    "Hey! How about we continue our English chat? What's new?",
    "Ready for another round of English practice? Ask me something!",
    "Let's keep the English flowing! What topic interests you today?",
    "Don't stop now! Send me another message or voice note in English. 😊"
]

async def scheduled_audio_reminder_task_v4(application: Application):

    try: TARGET_TIMEZONE_V4 = pytz.timezone('America/Sao_Paulo')
    except: TARGET_TIMEZONE_V4 = pytz.utc
    print(f"INFO: Audio Reminder Task V4 started (TZ: {TARGET_TIMEZONE_V4}).")
    while True:
        now_local = datetime.datetime.now(TARGET_TIMEZONE_V4)
        if 9 <= now_local.hour <= 19:
            await asyncio.sleep(random.randint(75*60, 180*60))
            current_local_after_sleep = datetime.datetime.now(TARGET_TIMEZONE_V4)
            if not (9 <= current_local_after_sleep.hour <= 19): continue
            if random.random() < 0.15 and schedulable_chat_ids_v4:
                target_chat_id = random.choice(list(schedulable_chat_ids_v4))
                tts_text = random.choice(reminder_messages_audio_en_v4)
                tts_file = await text_to_speech_v3(tts_text, target_chat_id, 'en-US', "reminder_audio_v4")
                if tts_file:
                    try:
                        await application.bot.send_voice(chat_id=target_chat_id, voice=open(tts_file, 'rb'))
                    except Exception as e:
                        print(f"SchedulerV4 Audio Error for {target_chat_id}: {e}")
                        if any(err_kw in str(e).lower() for err_kw in ["chat not found", "bot was blocked"]):
                            schedulable_chat_ids_v4.discard(target_chat_id)
                    finally:
                        if os.path.exists(tts_file): os.remove(tts_file)
        else:
            next_run = now_local.replace(hour=9, minute=0, second=0, microsecond=0)
            if now_local.hour > 19: next_run += datetime.timedelta(days=1)
            sleep_s = (next_run - now_local).total_seconds()
            if sleep_s <= 0: sleep_s = (next_run + datetime.timedelta(days=1) - now_local).total_seconds()
            if sleep_s <=0: sleep_s = 3600
            print(f"SchedulerV4 Audio: Off-hours. Sleeping for {sleep_s/3600:.1f}h.")
            await asyncio.sleep(max(60, sleep_s))


async def start_handler_v4(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user, chat_id = update.effective_user, update.effective_chat.id
    schedulable_chat_ids_v4.add(chat_id)
    welcome_msg = (f"Hello {user.mention_html()}! I'm <b>English Buddy</b> 🤖.\n\n"
                   "I auto-detect 🇬🇧 EN / 🇧🇷 PT. I'll reply in the same language and try to keep our chat going!\n"
                   "✍️ Text in ➔ Text out. 🎙️ Voice in ➔ Voice out.\n"
                   "I also send short English voice reminders (9am-7pm São Paulo time).\n\n"
                   "Type /help for a quick guide. Let's chat!")
    await update.message.reply_html(welcome_msg)
    if chat_id in chat_histories: del chat_histories[chat_id]
    print(f"CMD /start V4: User {user.id} (ChatID {chat_id}). Added to schedulable.")

async def help_handler_v4(update: Update, context: ContextTypes.DEFAULT_TYPE):
    help_msg = ("<b>English Buddy - How I Work</b> 🧐\n\n"
                "<code>/start</code> - Initialize/reset chat.\n"
                "<code>/help</code> - This guide.\n\n"
                "🗣️ **Auto Language & Engagement:** I reply in the language you use (EN/PT) and try to continue the conversation.\n"
                "➡️ **Input = Output Type:** Text in ➔ Text out. Voice in ➔ Voice out.\n"
                "My replies are short & clear. Just start chatting!")
    await update.message.reply_html(help_msg)

async def handle_user_interaction_v4(update: Update, context: ContextTypes.DEFAULT_TYPE, text: str, lang: str, is_voice_in: bool):
    chat_id = update.effective_chat.id
    schedulable_chat_ids_v4.add(chat_id)
    action = "record_voice" if is_voice_in else "typing"
    await context.bot.send_chat_action(chat_id=chat_id, action=action)


    resp_txt, tts_override, tts_lang = await get_gemini_response_v4(chat_id, text, lang)

    if is_voice_in:
        tts_speak_text = tts_override if tts_override else resp_txt
        tts_sotaque = "pt-BR" if tts_lang == "pt" else "en-US"
        if tts_speak_text and "AI Key missing" not in tts_speak_text and "error" not in tts_speak_text.lower():

            tts_f = await text_to_speech_v3(tts_speak_text, chat_id, tts_sotaque, "tts_audio_v4")
            if tts_f:
                try: await context.bot.send_voice(chat_id=chat_id, voice=open(tts_f, 'rb'))
                except Exception as e:
                    print(f"V4 Send Voice Error (ChatID {chat_id}): {e}")
                    await update.message.reply_text(f"(Audio send failed):\n{resp_txt}")
                finally:
                    if os.path.exists(tts_f): os.remove(tts_f)
            else: await update.message.reply_text(f"(TTS gen failed):\n{resp_txt}")
        else: await update.message.reply_text(resp_txt)
    else:
        await update.message.reply_text(resp_txt)

async def text_message_handler_v4(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_text = update.message.text
    pt_kws = [" o que ", " que ", " como ", " se ", " em ", " para ", " de ", " com ", " um ", " uma ", " eu ", " você "]
    en_kws = [" the ", " is ", " are ", " to ", " and ", " a ", " an ", " in ", " for ", " i ", " you "]
    pt_score = sum(1 for kw in pt_kws if kw in user_text.lower())
    en_score = sum(1 for kw in en_kws if kw in user_text.lower())
    lang = "pt" if pt_score > en_score + 1 else "en"
    if len(user_text.split()) < 3 and not pt_score and not en_score: lang = 'en'
    print(f"TEXT V4 (ChatID {update.effective_chat.id}, InferredLang {lang}): '{user_text[:50]}...'")
    await handle_user_interaction_v4(update, context, user_text, lang, is_voice_in=False)

async def voice_message_handler_v4(update: Update, context: ContextTypes.DEFAULT_TYPE):
    chat_id, voice = update.effective_chat.id, update.message.voice
    print(f"VOICE V4 (ChatID {chat_id}), Duration: {voice.duration}s")
    file_id = voice.file_id
    voice_f_obj = await context.bot.get_file(file_id)
    ogg_path = f"voice_in_v4_{chat_id}_{int(time.time())}.ogg"
    await voice_f_obj.download_to_drive(ogg_path)

    text, lang = await speech_to_text_v3(ogg_path, chat_id)
    if os.path.exists(ogg_path): os.remove(ogg_path)

    if not text or not lang:
        err_reply = "Sorry, couldn't understand your voice. Please try again."
        await update.message.reply_text(err_reply)
        tts_sotaque_err = 'pt-BR' if lang == 'pt' else 'en-US'

        tts_err_f = await text_to_speech_v3(err_reply, chat_id, tts_sotaque_err, "tts_error_v4")
        if tts_err_f:
            try: await context.bot.send_voice(chat_id=chat_id, voice=open(tts_err_f, 'rb'))
            finally:
                if os.path.exists(tts_err_f): os.remove(tts_err_f)
        return
    await handle_user_interaction_v4(update, context, text, lang, is_voice_in=True)

async def cleanup_temp_files_v4(directory="/content/"):
    count = 0

    for filename in os.listdir(directory):
        if (filename.startswith("tts_audio_v4_") and filename.endswith(".mp3")) or \
           (filename.startswith("tts_error_v4_") and filename.endswith(".mp3")) or \
           (filename.startswith("reminder_audio_v4_") and filename.endswith(".mp3")) or \
           (filename.startswith("voice_in_v4_") and filename.endswith(".ogg")):
            try:
                os.remove(os.path.join(directory, filename))
                count += 1
            except Exception as e: print(f"CleanupV4 Error: {e} on {filename}")



async def run_bot_v4():
    if not BOT_TOKEN_CONFIGURED or not API_KEY_CONFIGURED:
        print("FATAL: Bot/API keys missing. Exiting.")
        return
    if 'TELEGRAM_BOT_TOKEN' not in globals() or not TELEGRAM_BOT_TOKEN:
        print("FATAL: TELEGRAM_BOT_TOKEN not found globally. Exiting.")
        return

    app = Application.builder().token(TELEGRAM_BOT_TOKEN).build()
    app.add_handler(CommandHandler("start", start_handler_v4))
    app.add_handler(CommandHandler("help", help_handler_v4))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, text_message_handler_v4))
    app.add_handler(MessageHandler(filters.VOICE, voice_message_handler_v4))

    print("INFO: Telegram Bot V4 (Engaging, Auto-Lang, Concise) starting...")
    try:
        await app.initialize()
        asyncio.create_task(scheduled_audio_reminder_task_v4(app))
        await app.updater.start_polling()
        await app.start()
        print("INFO: Bot V4 running. Interrupt cell to stop.")
        while True: await asyncio.sleep(3600)
    except (KeyboardInterrupt, SystemExit): print("INFO: Bot V4 shutdown by user.")
    except Exception as e: print(f"CRITICAL: Bot V4 main loop error: {e}")
    finally:
        print("INFO: Shutting down Bot V4...")
        if hasattr(app, 'updater') and app.updater and app.updater.is_running: await app.updater.stop()

        if hasattr(app, 'stop_running') and callable(getattr(app, 'stop_running')) :
             app.stop_running()
        elif hasattr(app, 'stop') and callable(getattr(app, 'stop')):
            await app.stop()

        if hasattr(app, 'shutdown'): await app.shutdown()
        print("INFO: Bot V4 shutdown complete.")
        await cleanup_temp_files_v4()

In [24]:

import asyncio
import nest_asyncio

API_KEY_CONFIGURED_RUN = False
BOT_TOKEN_CONFIGURED_RUN = False
TELEGRAM_BOT_TOKEN_RUN = None

try:
    if 'os' not in globals(): import os
    if 'userdata' not in globals(): from google.colab import userdata
    if 'genai' not in globals(): import google.generativeai as genai

    _gkey = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=_gkey)
    API_KEY_CONFIGURED_RUN = True
except Exception:
    print("ERRO (Célula Exec): Falha ao configurar Gemini Key para execução.")

try:
    TELEGRAM_BOT_TOKEN_RUN = userdata.get('TELEGRAM_BOT_TOKEN')
    if not TELEGRAM_BOT_TOKEN_RUN: raise ValueError("Token Telegram vazio para execução.")
    BOT_TOKEN_CONFIGURED_RUN = True
except Exception:
     print("ERRO (Célula Exec): Falha ao carregar Token Telegram para execução.")



if API_KEY_CONFIGURED_RUN and BOT_TOKEN_CONFIGURED_RUN:
    print("INFO: Config V4 OK for execution. Starting Telegram bot V4...")
    nest_asyncio.apply()
    try:
        asyncio.run(run_bot_v4())
    except KeyboardInterrupt:
        print("\nINFO: Bot V4 execution interrupted by user from cell.")
    except RuntimeError as e:
        if "cannot schedule new futures after shutdown" in str(e):
            print("INFO: Bot V4 event loop already shut down.")
        else:
            print(f"CRITICAL: RuntimeError running Bot V4 cell: {e}")
    except Exception as e:
        print(f"CRITICAL: Error running Bot V4 cell: {e}")
    finally:
        print("INFO: Bot V4 execution cell finished.")
else:
    print("ERROR: Bot V4 cannot start. Check API/Token Keys in Cell 2 and Colab Secrets for this session.")

INFO: Config V4 OK for execution. Starting Telegram bot V4...

INFO: Bot V4 execution interrupted by user from cell.
INFO: Bot V4 execution cell finished.
